# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-12-04 11:32:02 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Imports

In [2]:
using LinearAlgebra
using Statistics
using Dates

# How to get all the dates corresponding to the month of July 2016?

In [3]:
dr = Date("2016-07-01"):Day(1):Date("2016-07-31")

Date("2016-07-01"):Day(1):Date("2016-07-31")

In [4]:
# The above is probably all you need but if if do need to get the dates individually then this works
dr = [Date("2016-07-01"):Day(1):Date("2016-07-31");]

31-element Vector{Date}:
 2016-07-01
 2016-07-02
 2016-07-03
 2016-07-04
 2016-07-05
 2016-07-06
 2016-07-07
 2016-07-08
 2016-07-09
 2016-07-10
 2016-07-11
 2016-07-12
 2016-07-13
 ⋮
 2016-07-20
 2016-07-21
 2016-07-22
 2016-07-23
 2016-07-24
 2016-07-25
 2016-07-26
 2016-07-27
 2016-07-28
 2016-07-29
 2016-07-30
 2016-07-31

# How to compute ((A+B)*(-A/2)) in place (without copy)?

In [5]:
A = ones(3)
B = 2ones(3)

(A + B) .* (-A / 2)

3-element Vector{Float64}:
 -1.5
 -1.5
 -1.5

# Extract the integer part of a random array of positive numbers?

In [6]:
Z = rand(0:0.01:100,20)
make_int(x) = trunc(Int,x)
make_int.(Z)

20-element Vector{Int64}:
 52
 79
 35
 81
 45
 93
 23
 74
  5
 74
 74
 96
 36
 98
 48
 20
 57
 57
 69
 76

# Create a 5x5 matrix with row values ranging from 0 to 4 

In [7]:
A = fill([0:4;], 5)
# This is the solution if you want a array of arrays

5-element Vector{Vector{Int64}}:
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]

In [8]:
# The solution for this particular problem
A = [0 1 2 3 4]
repeat(A, 5)

5×5 Matrix{Int64}:
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4

In [9]:
# This is a more generic solution but slightly more involved
repeat(reshape([0:4;], 1, :), 5)
# Lets break down as so 
# * [0:4;] creates a vector [0,1,2,3,4], by default Julia makes a column vector
# * reshape([0:4;], 1, :) converts to 1x5 row vector from the column vector
# * The repeat function basically copies it five times

5×5 Matrix{Int64}:
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4

# Create a vector of size 10 with values ranging from 0 to 1, both excluded


In [10]:
Z = [rand(Float64) for i in 1:10]
# Most of the time this will work. I don't really see the need to actually exclude 0 as well from this code but if you do, then

10-element Vector{Float64}:
 0.519234816475314
 0.1804018403309362
 0.2714710388811914
 0.35461677764030086
 0.08009917672939149
 0.9305222309809507
 0.28936220584069217
 0.48734320898935457
 0.2669961508581793
 0.7919648984699672

In [11]:
Z = [prevfloat(1.0) * (1 - rand(Float64)) for i in 1:10]
# Again its pretty extremely unlikely you'd actually need it but if you do, then you please read this discussion as well before going in https://discourse.julialang.org/t/how-to-create-a-random-uniform-distribution-between-but-excluding-0-and-10

10-element Vector{Float64}:
 0.11902292487378528
 0.39693991467233797
 0.8267670874453369
 0.6650298362130401
 0.09483749555455011
 0.005386419491226534
 0.5306254899868136
 0.6896840714116771
 0.8213758126865102
 0.4126411266446692

# Create a random vector of size 10 and sort it

In [12]:
Z = rand(Int,10)
sort!(Z)

10-element Vector{Int64}:
 -8411824846715949776
 -6326224349634966082
 -3490684567412628900
 -2913127372745197841
  1957564238359123547
  3727372096049137269
  4200831978661953860
  5285333814077719815
  5893104628279371421
  9144175940540116357

# Consider two random array A and B, check if they are equal

In [25]:
A = rand(1:5, 10)
B = rand(1:5, 10)

# To check if the arrays are completely equal
A == B

false

In [27]:
A = rand(1:5, 10)
B = rand(1:5, 10)
A,B

([5, 3, 2, 4, 1, 3, 4, 4, 4, 1], [1, 2, 4, 2, 5, 3, 4, 2, 3, 3])

In [29]:
# To check if the elements in the arrays are equal
A .== B

10-element BitVector:
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0

# Consider a random 10x2 matrix representing cartesian coordinates, convert them to polar coordinates


In [32]:
struct cartesian_point
    X::Float32
    Y::Float32
end
points = [cartesian_point(rand(-900:0.01:900), rand(-900:0.01:900)) for _ = 1:10]

10-element Vector{cartesian_point}:
 cartesian_point(-277.35f0, -792.09f0)
 cartesian_point(225.68f0, -57.51f0)
 cartesian_point(679.32f0, 523.94f0)
 cartesian_point(-696.6f0, 839.61f0)
 cartesian_point(-831.72f0, -657.35f0)
 cartesian_point(712.21f0, 220.42f0)
 cartesian_point(722.01f0, -607.81f0)
 cartesian_point(531.93f0, -763.26f0)
 cartesian_point(-323.1f0, 688.0f0)
 cartesian_point(-112.85f0, 29.74f0)

In [42]:
points = [cartesian_point(rand(), rand()) for _ = 1:10]
Xpoints = [point.X for point in points]
Ypoints = [point.Y for point in points]

10-element Vector{Float32}:
 0.30145374
 0.63208747
 0.5519679
 0.9418608
 0.033966314
 0.5044402
 0.21350227
 0.97808915
 0.5539718
 0.65762746

In [43]:
radians(x, y) = sqrt(x^2 + y^2)
theta(x, y) = atan(y, x)

theta (generic function with 1 method)

In [44]:
radians.(Xpoints, Ypoints)
theta.(Xpoints, Ypoints)

10-element Vector{Float32}:
 0.68233347
 1.3096839
 0.7383515
 0.85909283
 0.04828144
 0.5720019
 0.26204243
 0.9820394
 0.56690085
 0.89639544